In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import copy

import numpy as np
import torch
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

from src.data import load_data, transform_data, MyDataset
from src.attacks import attack_procedure
from src.utils import save_experiment, load_disc_model
from src.config import get_attack, load_disc_config

from src.models.LSTM import LSTM_net
from src.attacks import deepfool_attack

In [3]:
dataset = 'Ford_A_LSTM'

cfg = {
    'slice': True,
    'dataset': dataset,
    'batch_size': 1,
    'cuda': 'cuda:1',
    'train_mode': True,
    'model_folder': f'checkpoints/LSTM/{dataset}/',
    "model_id_attack": 0,
}

In [4]:
X_train, y_train, X_test, y_test = load_data(cfg['dataset'])
X_train, X_test, y_train, y_test = transform_data(X_train, X_test, y_train, y_test, slice_data=cfg['slice'])

test_loader = DataLoader(
    MyDataset(X_test, y_test), 
    batch_size=cfg['batch_size'] , 
    shuffle=True
    )

device= torch.device(cfg['cuda'] if torch.cuda.is_available() else 'cpu')

model = LSTM_net(
    hidden_dim = 50,
    n_layers = 1,
    output_dim = 1,
    dropout = 0.0,
).to(device)

model_path = cfg['model_folder'] + f'model_{cfg["model_id_attack"]}_{cfg["dataset"]}.pth'
model.load_state_dict(copy.deepcopy(torch.load(model_path)))

model.train(cfg['train_mode'])

LSTM_net(
  (rnn): LSTM(1, 50, batch_first=True)
  (fc1): Linear(in_features=50, out_features=50, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (tanh): Tanh()
)

# While loop

In [5]:
n_steps_max = 1000
eps = 1
log_iters = np.zeros(len(test_loader))

n_broken = 0

for i, (x, y_true) in tqdm(enumerate(test_loader)):
    #print(f'OBJECT {i}')
    y_true = y_true.to(device)
    y_pred = y_true.clone()
    x_adv = x.clone().to(device)

    x_adv.grad = None
    x_adv.requires_grad = True

    step = 0
    while step < n_steps_max:
        x_adv = deepfool_attack(model, None, x_adv, None, eps)
        y_pred = model(x_adv, use_sigmoid=False, use_tanh=True)
        step += 1

        status = 0
        # print(
        #     y_pred.detach().cpu(), 
        #     (y_true - 0.5).detach().cpu(), 
        #     (torch.sign(y_pred) != torch.sign(y_true - 0.5)).detach().cpu(), 
        # )

        if torch.sign(y_pred) != torch.sign(y_true - 0.5):
            #print(f'Object successful attacked with {step} iterations')
            status = 1
            n_broken += status
            log_iters[i] = step
            break
    else:
        print(f'for obj {i} iterations is over')


0it [00:00, ?it/s]

for obj 3 iterations is over
for obj 4 iterations is over
for obj 6 iterations is over
for obj 8 iterations is over
for obj 10 iterations is over
for obj 11 iterations is over
for obj 13 iterations is over
for obj 18 iterations is over
for obj 21 iterations is over
for obj 24 iterations is over
for obj 25 iterations is over
for obj 28 iterations is over
for obj 30 iterations is over
for obj 31 iterations is over
for obj 32 iterations is over
for obj 37 iterations is over
for obj 42 iterations is over
for obj 45 iterations is over
for obj 47 iterations is over
for obj 48 iterations is over
for obj 50 iterations is over
for obj 55 iterations is over
for obj 57 iterations is over
for obj 59 iterations is over
for obj 66 iterations is over
for obj 72 iterations is over
for obj 75 iterations is over
for obj 81 iterations is over
for obj 85 iterations is over
for obj 86 iterations is over
for obj 87 iterations is over
for obj 88 iterations is over
for obj 89 iterations is over
for obj 94 ite

KeyboardInterrupt: 

In [6]:
print(f'Accuracy {round(n_broken / len(test_loader), 7)}')

Accuracy 0.6985606


In [7]:
print(f'Accuracy {round(n_broken / 5323, 5)}')

Accuracy 0.6966


# iterations -> loader

In [5]:
n_steps_max = 50
eps = 1
log_iters = np.zeros(len(test_loader))

n_broken = 0

for i, (x, y_true) in tqdm(enumerate(test_loader)):
    # print(f'OBJECT {i}')
    y_true = y_true.to(device)
    y_pred = y_true.clone()
    x_adv = x.clone().to(device)

    x_adv.grad = None
    x_adv.requires_grad = True

    step = 0
    while step < n_steps_max:
        x_adv = deepfool_attack(model, None, x_adv, None, eps)
        y_pred = model(x_adv)
        step += 1

        status = 0
        # print(
        #     y_pred.detach().cpu(), 
        #     torch.round(y_pred).detach().cpu(), 
        #     y_true.detach().cpu(), 
        #     (torch.round(y_pred) != y_true).detach().cpu()
        # )
        if torch.round(y_pred) != y_true:
            #print(f'Object successful attacked with {step} iterations')
            status = 1
            #n_broken += status
            log_iters[i] = step
            #break
    else:
        n_broken += status
        #print(f'for obj {i} iterations is over')


In [6]:
print(f'Accuracy {round(n_broken / len(test_loader), 7)}')

Accuracy 0.4905303
